# Baixando e Instalando Bibliotecas e Base de Dados para o Estudo de Caso.
***

In [ ]:
#!pip install --upgrade pip
#!sudo apt install libffi-dev
#!sudo pip install PySUS

In [ ]:
#from pysus.online_data.sinasc import download, get_available_years # Base de Dados
#import pysus
import matplotlib.pyplot as plt# Biblioteca de gráficos
import seaborn as sns# Biblioteca de gráficos
import pandas as pd # Biblioteca de gerencaimento de tabeals
import numpy as np # Biblioteca de Vetores
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures # Biblioteca para tratameto de objetos
from sklearn.impute import SimpleImputer # Completar valores faltantes da tabela
from sklearn.compose import ColumnTransformer # Transormador de Colunas do Sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from google.colab import drive # Biblioteca para acessar o drive
from scipy import stats # Biblioteca para Estatística

drive.mount('/content/drive')
pd.set_option("display.max_columns", None) # Configurand para mostrar todas as colunas

Mounted at /content/drive


ipeia


#Machine Learning
***

## Objetivo e Base de Dados

O objetivo finál deste estudo de caso é desenvollver um algoritmo treinado de machine learning para previsão da probabilidade de cada parto baseado em índices soxioeconomicos.
Como o SINASC disponibiliza o registro de múltiplos anos e cada tabela contém de 300 mil a 600 mil registros, utilizar a junção de todos os anos é inviável, pois o treinamento demoraria muito tempo. Más criando um dataframe com uma pequena parte de todas as tabelas, possívelmente será obtido a variabilidade de dados e um tempo de treinamento adequado.

In [ ]:
SP = pd.read_csv('/content/drive/MyDrive/SINASC/Data/SP/SP_2012_2020.csv').drop('Unnamed: 0', axis=1)
SP

,PARTO,PESO,SEMAGESTAC,STTRABPART,DTULTMENST,IDADEMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,QTDFILVIVO,QTDFILMORT,CODMUNRES_350000,CODMUNRES_350010,CODMUNRES_350020,CODMUNRES_350030,CODMUNRES_350040,CODMUNRES_350050,CODMUNRES_350055,CODMUNRES_350060,CODMUNRES_350070,CODMUNRES_350075,CODMUNRES_350080,CODMUNRES_350090,CODMUNRES_350100,CODMUNRES_350110,CODMUNRES_350115,CODMUNRES_350120,CODMUNRES_350130,CODMUNRES_350140,CODMUNRES_350150,CODMUNRES_350160,CODMUNRES_350170,CODMUNRES_350180,CODMUNRES_350190,CODMUNRES_350200,CODMUNRES_350210,CODMUNRES_350220,CODMUNRES_350230,CODMUNRES_350240,CODMUNRES_350250,CODMUNRES_350260,CODMUNRES_350270,CODMUNRES_350275,CODMUNRES_350280,CODMUNRES_350290,CODMUNRES_350300,CODMUNRES_350310,CODMUNRES_350315,CODMUNRES_350320,CODMUNRES_350330,CODMUNRES_350335,CODMUNRES_350340,CODMUNRES_350350,CODMUNRES_350360,CODMUNRES_350370,CODMUNRES_350380,CODMUNRES_350390,CODMUNRES_350395,CODMUNRES_350400,CODMUNRES_350410,CODMUNRES_350420,CODMUNRES_350430,CODMUNRES_350440,CODMUNRES_350450,CODMUNRES_350460,CODMUNRES_350470,CODMUNRES_350480,CODMUNRES_350490,CODMUNRES_350500,CODMUNRES_350510,CODMUNRES_350520,CODMUNRES_350530,CODMUNRES_350535,CODMUNRES_350540,CODMUNRES_350550,CODMUNRES_350560,CODMUNRES_350570,CODMUNRES_350580,CODMUNRES_350590,CODMUNRES_350600,CODMUNRES_350610,CODMUNRES_350620,CODMUNRES_350630,CODMUNRES_350635,CODMUNRES_350640,CODMUNRES_350650,CODMUNRES_350660,CODMUNRES_350670,CODMUNRES_350680,CODMUNRES_350690,CODMUNRES_350700,CODMUNRES_350710,CODMUNRES_350715,CODMUNRES_350720,CODMUNRES_350730,CODMUNRES_350740,CODMUNRES_350745,CODMUNRES_350750,CODMUNRES_350760,CODMUNRES_350770,CODMUNRES_350775,CODMUNRES_350780,CODMUNRES_350790,CODMUNRES_350800,CODMUNRES_350810,CODMUNRES_350820,CODMUNRES_350830,CODMUNRES_350840,CODMUNRES_350850,CODMUNRES_350860,CODMUNRES_350870,CODMUNRES_350880,CODMUNRES_350890,CODMUNRES_350900,CODMUNRES_350910,CODMUNRES_350920,CODMUNRES_350925,CODMUNRES_350930,CODMUNRES_350940,CODMUNRES_350945,CODMUNRES_350950,CODMUNRES_350960,CODMUNRES_350970,CODMUNRES_350980,CODMUNRES_350990,CODMUNRES_350995,CODMUNRES_351000,CODMUNRES_351010,CODMUNRES_351015,CODMUNRES_351020,CODMUNRES_351030,CODMUNRES_351040,CODMUNRES_351050,CODMUNRES_351060,CODMUNRES_351070,CODMUNRES_351080,CODMUNRES_351090,CODMUNRES_351100,CODMUNRES_351110,CODMUNRES_351120,CODMUNRES_351130,CODMUNRES_351140,CODMUNRES_351150,CODMUNRES_351160,CODMUNRES_351170,CODMUNRES_351190,CODMUNRES_351200,CODMUNRES_351210,CODMUNRES_351220,CODMUNRES_351230,CODMUNRES_351240,CODMUNRES_351250,CODMUNRES_351260,CODMUNRES_351270,CODMUNRES_351280,CODMUNRES_351290,CODMUNRES_351300,CODMUNRES_351310,CODMUNRES_351320,CODMUNRES_351330,CODMUNRES_351340,CODMUNRES_351350,CODMUNRES_351360,CODMUNRES_351370,CODMUNRES_351380,CODMUNRES_351385,CODMUNRES_351390,CODMUNRES_351400,CODMUNRES_351410,CODMUNRES_351420,CODMUNRES_351430,CODMUNRES_351440,CODMUNRES_351450,CODMUNRES_351460,CODMUNRES_351470,CODMUNRES_351480,CODMUNRES_351490,CODMUNRES_351495,CODMUNRES_351500,CODMUNRES_351510,CODMUNRES_351512,CODMUNRES_351515,CODMUNRES_351518,CODMUNRES_351519,CODMUNRES_351520,CODMUNRES_351530,CODMUNRES_351535,CODMUNRES_351540,CODMUNRES_351550,CODMUNRES_351560,CODMUNRES_351565,CODMUNRES_351570,CODMUNRES_351580,CODMUNRES_351590,CODMUNRES_351600,CODMUNRES_351610,CODMUNRES_351620,CODMUNRES_351630,CODMUNRES_351640,CODMUNRES_351650,CODMUNRES_351660,CODMUNRES_351670,CODMUNRES_351680,CODMUNRES_351685,CODMUNRES_351690,CODMUNRES_351700,CODMUNRES_351710,CODMUNRES_351720,CODMUNRES_351730,CODMUNRES_351740,CODMUNRES_351750,CODMUNRES_351760,CODMUNRES_351770,CODMUNRES_351780,CODMUNRES_351790,CODMUNRES_351810,CODMUNRES_351820,CODMUNRES_351830,CODMUNRES_351840,CODMUNRES_351850,CODMUNRES_351860,CODMUNRES_351870,CODMUNRES_351880,CODMUNRES_351885,CODMUNRES_351890,CODMUNRES_351900,CODMUNRES_351905,CODMUNRES_351907,CODMUNRES_351910,CODMUNRES_351920,CODMUNRES_351925,CODMUNRES_351930,CODMUNRES_351940,CODMUNRES_351950,CODMUNRES_351960,CODMUNRES_351970,CODMUNRES_351980,CODMUNRES_351990,CODMUNRES_352000,CODMUNRES_35201

In [ ]:
X = SP.drop('PARTO', axis=1)
X

,PESO,SEMAGESTAC,STTRABPART,DTULTMENST,IDADEMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,QTDFILVIVO,QTDFILMORT,CODMUNRES_350000,CODMUNRES_350010,CODMUNRES_350020,CODMUNRES_350030,CODMUNRES_350040,CODMUNRES_350050,CODMUNRES_350055,CODMUNRES_350060,CODMUNRES_350070,CODMUNRES_350075,CODMUNRES_350080,CODMUNRES_350090,CODMUNRES_350100,CODMUNRES_350110,CODMUNRES_350115,CODMUNRES_350120,CODMUNRES_350130,CODMUNRES_350140,CODMUNRES_350150,CODMUNRES_350160,CODMUNRES_350170,CODMUNRES_350180,CODMUNRES_350190,CODMUNRES_350200,CODMUNRES_350210,CODMUNRES_350220,CODMUNRES_350230,CODMUNRES_350240,CODMUNRES_350250,CODMUNRES_350260,CODMUNRES_350270,CODMUNRES_350275,CODMUNRES_350280,CODMUNRES_350290,CODMUNRES_350300,CODMUNRES_350310,CODMUNRES_350315,CODMUNRES_350320,CODMUNRES_350330,CODMUNRES_350335,CODMUNRES_350340,CODMUNRES_350350,CODMUNRES_350360,CODMUNRES_350370,CODMUNRES_350380,CODMUNRES_350390,CODMUNRES_350395,CODMUNRES_350400,CODMUNRES_350410,CODMUNRES_350420,CODMUNRES_350430,CODMUNRES_350440,CODMUNRES_350450,CODMUNRES_350460,CODMUNRES_350470,CODMUNRES_350480,CODMUNRES_350490,CODMUNRES_350500,CODMUNRES_350510,CODMUNRES_350520,CODMUNRES_350530,CODMUNRES_350535,CODMUNRES_350540,CODMUNRES_350550,CODMUNRES_350560,CODMUNRES_350570,CODMUNRES_350580,CODMUNRES_350590,CODMUNRES_350600,CODMUNRES_350610,CODMUNRES_350620,CODMUNRES_350630,CODMUNRES_350635,CODMUNRES_350640,CODMUNRES_350650,CODMUNRES_350660,CODMUNRES_350670,CODMUNRES_350680,CODMUNRES_350690,CODMUNRES_350700,CODMUNRES_350710,CODMUNRES_350715,CODMUNRES_350720,CODMUNRES_350730,CODMUNRES_350740,CODMUNRES_350745,CODMUNRES_350750,CODMUNRES_350760,CODMUNRES_350770,CODMUNRES_350775,CODMUNRES_350780,CODMUNRES_350790,CODMUNRES_350800,CODMUNRES_350810,CODMUNRES_350820,CODMUNRES_350830,CODMUNRES_350840,CODMUNRES_350850,CODMUNRES_350860,CODMUNRES_350870,CODMUNRES_350880,CODMUNRES_350890,CODMUNRES_350900,CODMUNRES_350910,CODMUNRES_350920,CODMUNRES_350925,CODMUNRES_350930,CODMUNRES_350940,CODMUNRES_350945,CODMUNRES_350950,CODMUNRES_350960,CODMUNRES_350970,CODMUNRES_350980,CODMUNRES_350990,CODMUNRES_350995,CODMUNRES_351000,CODMUNRES_351010,CODMUNRES_351015,CODMUNRES_351020,CODMUNRES_351030,CODMUNRES_351040,CODMUNRES_351050,CODMUNRES_351060,CODMUNRES_351070,CODMUNRES_351080,CODMUNRES_351090,CODMUNRES_351100,CODMUNRES_351110,CODMUNRES_351120,CODMUNRES_351130,CODMUNRES_351140,CODMUNRES_351150,CODMUNRES_351160,CODMUNRES_351170,CODMUNRES_351190,CODMUNRES_351200,CODMUNRES_351210,CODMUNRES_351220,CODMUNRES_351230,CODMUNRES_351240,CODMUNRES_351250,CODMUNRES_351260,CODMUNRES_351270,CODMUNRES_351280,CODMUNRES_351290,CODMUNRES_351300,CODMUNRES_351310,CODMUNRES_351320,CODMUNRES_351330,CODMUNRES_351340,CODMUNRES_351350,CODMUNRES_351360,CODMUNRES_351370,CODMUNRES_351380,CODMUNRES_351385,CODMUNRES_351390,CODMUNRES_351400,CODMUNRES_351410,CODMUNRES_351420,CODMUNRES_351430,CODMUNRES_351440,CODMUNRES_351450,CODMUNRES_351460,CODMUNRES_351470,CODMUNRES_351480,CODMUNRES_351490,CODMUNRES_351495,CODMUNRES_351500,CODMUNRES_351510,CODMUNRES_351512,CODMUNRES_351515,CODMUNRES_351518,CODMUNRES_351519,CODMUNRES_351520,CODMUNRES_351530,CODMUNRES_351535,CODMUNRES_351540,CODMUNRES_351550,CODMUNRES_351560,CODMUNRES_351565,CODMUNRES_351570,CODMUNRES_351580,CODMUNRES_351590,CODMUNRES_351600,CODMUNRES_351610,CODMUNRES_351620,CODMUNRES_351630,CODMUNRES_351640,CODMUNRES_351650,CODMUNRES_351660,CODMUNRES_351670,CODMUNRES_351680,CODMUNRES_351685,CODMUNRES_351690,CODMUNRES_351700,CODMUNRES_351710,CODMUNRES_351720,CODMUNRES_351730,CODMUNRES_351740,CODMUNRES_351750,CODMUNRES_351760,CODMUNRES_351770,CODMUNRES_351780,CODMUNRES_351790,CODMUNRES_351810,CODMUNRES_351820,CODMUNRES_351830,CODMUNRES_351840,CODMUNRES_351850,CODMUNRES_351860,CODMUNRES_351870,CODMUNRES_351880,CODMUNRES_351885,CODMUNRES_351890,CODMUNRES_351900,CODMUNRES_351905,CODMUNRES_351907,CODMUNRES_351910,CODMUNRES_351920,CODMUNRES_351925,CODMUNRES_351930,CODMUNRES_351940,CODMUNRES_351950,CODMUNRES_351960,CODMUNRES_351970,CODMUNRES_351980,CODMUNRES_351990,CODMUNRES_352000,CODMUNRES_352010,CODM

In [ ]:
y = SP['PARTO']
y

0        0.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
89995    0.0
89996    0.0
89997    0.0
89998    1.0
89999    0.0
Name: PARTO, Length: 90000, dtype: float64

## Pipelines para Organização

Inicialmente será criado um pipeline com as transformações da base de dados, após isso este mesmo pipeline poderá ser utilizado em diversos algoritmos diferentes. 

## Regressão Logística Linear

Como o problema inicil gira em torno da previsão entre a via de parto normal e cesário, logo este é um problema de regressão logística, a qual retorna a probabilidade de um vetor pertencer a uma classe. É possível utilizar como base da regressã logística a regressão linear ou polinomial.
As colunas categóricas são transformadas em colunas com os valores 0 ou 1, números que ao serem elevados a expoentes positívosresultam nno mesmo número, então apenas as colunas numéricas utilizam parametros polinomiais.
Como foi mostrado nas tabelas abaixo, houve pouca mudança na acurácia de parametros lineares ou polinomial, cerca de 0.5%.

In [ ]:
esc    = ['ESCMAE2010_0.0', 'ESCMAE2010_1.0', 'ESCMAE2010_2.0', 'ESCMAE2010_3.0', 'ESCMAE2010_4.0', 'ESCMAE2010_5.0']
est    = ['ESTCIVMAE_1.0', 'ESTCIVMAE_2.0', 'ESTCIVMAE_3.0', 'ESTCIVMAE_4.0', 'ESTCIVMAE_5.0']
raca   = ['RACACORMAE_1.0', 'RACACORMAE_2.0', 'RACACORMAE_3.0', 'RACACORMAE_4.0','RACACORMAE_5.0']

X_drop = []

for c in range(6):
  if c == 5:
    X_drop.append(X.drop([esc[c], est[c-1], raca[c-1]], axis=1))
  else:
    X_drop.append(X.drop([esc[c], est[c], raca[c]], axis=1))

X_drop[0]

,IDADEMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,QTDFILVIVO,QTDFILMORT,ESCMAE2010_1.0,ESCMAE2010_2.0,ESCMAE2010_3.0,ESCMAE2010_4.0,ESCMAE2010_5.0,ESTCIVMAE_2.0,ESTCIVMAE_3.0,ESTCIVMAE_4.0,ESTCIVMAE_5.0,RACACORMAE_2.0,RACACORMAE_3.0,RACACORMAE_4.0,RACACORMAE_5.0
0,30.0,1.0,0.0,0.0,0.0,1.0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,34.0,2.0,2.0,0.0,2.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,35.0,3.0,0.0,1.0,1.0,2.0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,3.0,2.0,1.0,3.0,0.0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,28.0,1.0,1.0,0.0,1.0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,36.0,2.0,0.0,1.0,1.0,1.0,0,0,0,0,1,1,0,0,0,0,0,1,0
89996,33.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0
89997,30.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0
89998,25.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

lista = []

for c in range(len(X_drop)):

  scaler = StandardScaler()
  X_ = scaler.fit_transform(X_drop[c])
  X_train, X_test, y_train, y_test = train_test_split(X_, y.to_numpy(), test_size=0.3, random_state=42) 

  model = sm.Logit(y_train, X_train, missing = 'raise')
  results =  model.fit()
  pvalue = results.pvalues
  coluna = X_drop[c].columns
  odds_ratio = np.exp(results.params)
  dfs = pd.DataFrame({
        f'colunas_{c}': coluna,
        f'odds_{c}': odds_ratio,
        f'pvalue_{c}': pvalue,
        })
  
  lista.append(dfs)


resultados = pd.concat(lista, axis=1)
resultados = resultados[6:].reset_index(drop=True)
resultados

Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548480
         Iterations 6


,colunas_0,odds_0,pvalue_0,colunas_1,odds_1,pvalue_1,colunas_2,odds_2,pvalue_2,colunas_3,odds_3,pvalue_3,colunas_4,odds_4,pvalue_4,colunas_5,odds_5,pvalue_5
0,ESCMAE2010_1.0,1.046090,2.988712e-01,ESCMAE2010_0.0,0.988665,2.988712e-01,ESCMAE2010_0.0,0.980152,6.053761e-02,ESCMAE2010_0.0,0.983995,1.300267e-01,ESCMAE2010_0.0,0.997359,8.056885e-01,ESCMAE2010_0.0,0.994315,5.938805e-01
1,ESCMAE2010_2.0,1.229889,6.053761e-02,ESCMAE2010_2.0,1.093363,2.333147e-03,ESCMAE2010_1.0,0.966396,2.333147e-03,ESCMAE2010_1.0,0.981458,8.521214e-02,ESCMAE2010_1.0,1.035211,4.737656e-03,ESCMAE2010_1.0,1.022781,4.797336e-02
2,ESCMAE2010_3.0,1.245051,1.300267e-01,ESCMAE2010_3.0,1.066433,8.521214e-02,ESCMAE2010_3.0,0.948237,1.049065e-04,ESCMAE2010_2.0,1.041213,1.049065e-04,ESCMAE2010_2.0,1.196767,3.083170e-23,ESCMAE2010_2.0,1.159606,1.114289e-25
3,ESCMAE2010_4.0,1.015913,8.056885e-01,ESCMAE2010_4.0,0.949085,4.737656e-03,ESCMAE2010_4.0,0.901339,3.083170e-23,ESCMAE2010_4.0,0.922637,1.798735e-18,ESCMAE2010_3.0,1.201111,1.798735e-18,ESCMAE2010_3.0,1.152269,8.338324e-25
4,ESCMAE2010_5.0,1.063315,5.938805e-01,ESCMAE2010_5.0,0.940477,4.797336e-02,ESCMAE2010_5.0,0.856852,1.114289e-25,ESCMAE2010_5.0,0.893726,8.338324e-25,ESCMAE2010_5.0,1.033457,7.180434e-02,ESCMAE2010_4.0,0.981924,7.180434e-02
5,ESTCIVMAE_2.0,0.856457,1.310245e-45,ESTCIVMAE_1.0,1.166992,1.310245e-45,ESTCIVMAE_1.0,1.214213,1.352128e-01,ESTCIVMAE_1.0,1.103897,7.487667e-03,ESTCIVMAE_1.0,0.967891,1.281854e-02,ESTCIVMAE_1.0,0.967891,1.281854e-02
6,ESTCIVMAE_3.0,0.984670,1.352128e-01,ESTCIVMAE_3.0,0.996848,7.600749e-01,ESTCIVMAE_2.0,1.040603,7.600749e-01,ESTCIVMAE_2.0,0.945756,1.299299e-01,ESTCIVMAE_2.0,0.828866,6.982092e-43,ESTCIVMAE_2.0,0.828866,6.982092e-43
7,ESTCIVMAE_4.0,0.973534,7.487667e-03,ESTCIVMAE_4.0,1.015197,1.299299e-01,ESTCIVMAE_4.0,1.026183,4.788985e-01,ESTCIVMAE_3.0,0.992447,4.788985e-01,ESTCIVMAE_3.0,0.982115,8.152923e-02,ESTCIVMAE_3.0,0.982115,8.152923e-02
8,ESTCIVMAE_5.0,1.025520,1.281854e-02,ESTCIVMAE_5.0,1.155393,6.982092e-43,ESTCIVMAE_5.0,1.191328,8.152923e-02,ESTCIVMAE_5.0,1.106856,5.007717e-04,ESTCIVMAE_4.0,0.964951,5.007717e-04,ESTCIVMAE_4.0,0.964951,5.007717e-04
9,RACACORMAE_2.0,1.057793,3.832101e-09,RACACORMAE_1.0,0.888695,3.832101e-09,RACACORMAE_1.0,0.797866,1.735872e-04,RACACORMAE_1.0,0.849937,3.028580e-59,RACACORMAE_1.0,0.677096,1.253984e-03,RACACORMAE_1.0,0.677096,1.253984e-03


In [ ]:
from sklearn.ensemble import RandomForestClassifier



model = RandomForestClassifier(max_depth=92, random_state=0) # Definindo modelo



for c in range(len(X_drop)):
  X_train, X_test, y_train, y_test = train_test_split(X_drop[c], y, test_size=0.3, random_state=42) 
  model = LogisticRegression() # Definindo modelo
  model.fit(X_train, y_train) # Treinando o modelo com os Dados de X_test e y_test
  print(model.score(X_test, y_test))






#print('Matriz de Confusão')
#plot_confusion_matrix(model, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7404444444444445


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7406666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7409629629629629


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7402592592592593
0.7399259259259259


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 


model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, 
                                   max_depth=10, random_state=0) # Definindo modelo
for c in range(len(X_drop)):
  X_train, X_test, y_train, y_test = train_test_split(X_drop[c], y, test_size=0.3, random_state=42) 
  model = LogisticRegression() # Definindo modelo
  model.fit(X_train, y_train) # Treinando o modelo com os Dados de X_test e y_test
  print(model.score(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7404444444444445


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7406666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7409629629629629


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7402592592592593
0.7399259259259259


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
X = X.drop(['STTRABPART'], axis=1)
X

,DTULTMENST,IDADEMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,QTDFILVIVO,QTDFILMORT,CODMUNRES_350000,CODMUNRES_350010,CODMUNRES_350020,CODMUNRES_350030,CODMUNRES_350040,CODMUNRES_350050,CODMUNRES_350055,CODMUNRES_350060,CODMUNRES_350070,CODMUNRES_350075,CODMUNRES_350080,CODMUNRES_350090,CODMUNRES_350100,CODMUNRES_350110,CODMUNRES_350115,CODMUNRES_350120,CODMUNRES_350130,CODMUNRES_350140,CODMUNRES_350150,CODMUNRES_350160,CODMUNRES_350170,CODMUNRES_350180,CODMUNRES_350190,CODMUNRES_350200,CODMUNRES_350210,CODMUNRES_350220,CODMUNRES_350230,CODMUNRES_350240,CODMUNRES_350250,CODMUNRES_350260,CODMUNRES_350270,CODMUNRES_350275,CODMUNRES_350280,CODMUNRES_350290,CODMUNRES_350300,CODMUNRES_350310,CODMUNRES_350315,CODMUNRES_350320,CODMUNRES_350330,CODMUNRES_350335,CODMUNRES_350340,CODMUNRES_350350,CODMUNRES_350360,CODMUNRES_350370,CODMUNRES_350380,CODMUNRES_350390,CODMUNRES_350395,CODMUNRES_350400,CODMUNRES_350410,CODMUNRES_350420,CODMUNRES_350430,CODMUNRES_350440,CODMUNRES_350450,CODMUNRES_350460,CODMUNRES_350470,CODMUNRES_350480,CODMUNRES_350490,CODMUNRES_350500,CODMUNRES_350510,CODMUNRES_350520,CODMUNRES_350530,CODMUNRES_350535,CODMUNRES_350540,CODMUNRES_350550,CODMUNRES_350560,CODMUNRES_350570,CODMUNRES_350580,CODMUNRES_350590,CODMUNRES_350600,CODMUNRES_350610,CODMUNRES_350620,CODMUNRES_350630,CODMUNRES_350635,CODMUNRES_350640,CODMUNRES_350650,CODMUNRES_350660,CODMUNRES_350670,CODMUNRES_350680,CODMUNRES_350690,CODMUNRES_350700,CODMUNRES_350710,CODMUNRES_350715,CODMUNRES_350720,CODMUNRES_350730,CODMUNRES_350740,CODMUNRES_350745,CODMUNRES_350750,CODMUNRES_350760,CODMUNRES_350770,CODMUNRES_350775,CODMUNRES_350780,CODMUNRES_350790,CODMUNRES_350800,CODMUNRES_350810,CODMUNRES_350820,CODMUNRES_350830,CODMUNRES_350840,CODMUNRES_350850,CODMUNRES_350860,CODMUNRES_350870,CODMUNRES_350880,CODMUNRES_350890,CODMUNRES_350900,CODMUNRES_350910,CODMUNRES_350920,CODMUNRES_350925,CODMUNRES_350930,CODMUNRES_350940,CODMUNRES_350945,CODMUNRES_350950,CODMUNRES_350960,CODMUNRES_350970,CODMUNRES_350980,CODMUNRES_350990,CODMUNRES_350995,CODMUNRES_351000,CODMUNRES_351010,CODMUNRES_351015,CODMUNRES_351020,CODMUNRES_351030,CODMUNRES_351040,CODMUNRES_351050,CODMUNRES_351060,CODMUNRES_351070,CODMUNRES_351080,CODMUNRES_351090,CODMUNRES_351100,CODMUNRES_351110,CODMUNRES_351120,CODMUNRES_351130,CODMUNRES_351140,CODMUNRES_351150,CODMUNRES_351160,CODMUNRES_351170,CODMUNRES_351190,CODMUNRES_351200,CODMUNRES_351210,CODMUNRES_351220,CODMUNRES_351230,CODMUNRES_351240,CODMUNRES_351250,CODMUNRES_351260,CODMUNRES_351270,CODMUNRES_351280,CODMUNRES_351290,CODMUNRES_351300,CODMUNRES_351310,CODMUNRES_351320,CODMUNRES_351330,CODMUNRES_351340,CODMUNRES_351350,CODMUNRES_351360,CODMUNRES_351370,CODMUNRES_351380,CODMUNRES_351385,CODMUNRES_351390,CODMUNRES_351400,CODMUNRES_351410,CODMUNRES_351420,CODMUNRES_351430,CODMUNRES_351440,CODMUNRES_351450,CODMUNRES_351460,CODMUNRES_351470,CODMUNRES_351480,CODMUNRES_351490,CODMUNRES_351495,CODMUNRES_351500,CODMUNRES_351510,CODMUNRES_351512,CODMUNRES_351515,CODMUNRES_351518,CODMUNRES_351519,CODMUNRES_351520,CODMUNRES_351530,CODMUNRES_351535,CODMUNRES_351540,CODMUNRES_351550,CODMUNRES_351560,CODMUNRES_351565,CODMUNRES_351570,CODMUNRES_351580,CODMUNRES_351590,CODMUNRES_351600,CODMUNRES_351610,CODMUNRES_351620,CODMUNRES_351630,CODMUNRES_351640,CODMUNRES_351650,CODMUNRES_351660,CODMUNRES_351670,CODMUNRES_351680,CODMUNRES_351685,CODMUNRES_351690,CODMUNRES_351700,CODMUNRES_351710,CODMUNRES_351720,CODMUNRES_351730,CODMUNRES_351740,CODMUNRES_351750,CODMUNRES_351760,CODMUNRES_351770,CODMUNRES_351780,CODMUNRES_351790,CODMUNRES_351810,CODMUNRES_351820,CODMUNRES_351830,CODMUNRES_351840,CODMUNRES_351850,CODMUNRES_351860,CODMUNRES_351870,CODMUNRES_351880,CODMUNRES_351885,CODMUNRES_351890,CODMUNRES_351900,CODMUNRES_351905,CODMUNRES_351907,CODMUNRES_351910,CODMUNRES_351920,CODMUNRES_351925,CODMUNRES_351930,CODMUNRES_351940,CODMUNRES_351950,CODMUNRES_351960,CODMUNRES_351970,CODMUNRES_351980,CODMUNRES_351990,CODMUNRES_352000,CODMUNRES_352010,CODMUNRES_352020,CODMUNRES_3520

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

esc    = ['ESCMAE2010_0.0', 'ESCMAE2010_1.0', 'ESCMAE2010_2.0', 'ESCMAE2010_3.0', 'ESCMAE2010_4.0', 'ESCMAE2010_5.0']
est    = ['ESTCIVMAE_1.0', 'ESTCIVMAE_2.0', 'ESTCIVMAE_3.0', 'ESTCIVMAE_4.0', 'ESTCIVMAE_5.0']
raca   = ['RACACORMAE_1.0', 'RACACORMAE_2.0', 'RACACORMAE_3.0', 'RACACORMAE_4.0','RACACORMAE_5.0']

i = 0
list_coefs = list()

for i in range(6):
    if i < 5:
      X_p = X.drop([esc[i], est[i], raca[i]], axis=1)
    else:
      X_p = X.drop([esc[i], est[i-1], raca[i-1]], axis=1)

    columns = X_p.columns
    X_p = StandardScaler().fit_transform(X_p)
    X_train, X_test, y_train, y_test = train_test_split(X_p, y.to_numpy(), test_size=0.3, random_state=42) 
    model = sm.Logit(y_train, X_train).fit()

    coefs = pd.DataFrame({
    f'columns': columns,
    'odds ratio': np.exp(model.params),
    'pvalue': model.pvalues,
    })
    list_coefs.append(coefs)
    i += 1

resultados = pd.concat(list_coefs, axis=1)

In [ ]:
import tensorflow as tf



model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(X.shape[1], )),
tf.keras.layers.Dense(300, activation='relu'),
tf.keras.layers.Dense(150, activation='relu'),
tf.keras.layers.Dense(50, activation='sigmoid'),
tf.keras.layers.Dense(2, activation='softmax')])

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
model.fit(X_train, y_train, epochs=15)
print('/n/n')
model.evaluate(X_test, y_test)

Epoch 1/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.5099 - accuracy: 0.7529
Epoch 2/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4847 - accuracy: 0.7681
Epoch 3/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4755 - accuracy: 0.7732
Epoch 4/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4681 - accuracy: 0.7773
Epoch 5/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4612 - accuracy: 0.7812
Epoch 6/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4552 - accuracy: 0.7845
Epoch 7/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4473 - accuracy: 0.7890
Epoch 8/15
2250/2250 [==============================] - 10s 4ms/step - loss: 0.4407 - accuracy: 0.7922
Epoch 9/15
2250/2250 [==============================] - 12s 5ms/step - loss: 0.4315 - accuracy: 0.7983
Epoch 10/15
2250/2250 [==============================] - 10s 4ms/step - l

[0.5406009554862976, 0.7648888826370239]

563/563 [==============================] - 2s 3ms/step - loss: 1.0700 - accuracy: 0.7358


[1.0699833631515503, 0.7357777953147888]

## Suporte Vetor Máquina

In [ ]:
from sklearn.svm import SVC


for c in range(len(X_drop)):
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
  modelo = SVC(kernel='linear', gamma=0.1, C=10)
  pipe = Pipeline(steps=[ ("scale", StandardScaler()),
                              ("modelo", modelo) ])
  pipe.fit(X_train, y_train)
  pipe.score(X_test, y_test)

In [ ]:
print(pipe.score(X_test, y_test))

0.7380740740740741


1) Seleção das variáveis:
Utilizar apenas aquelas que podem ser conhecidas antes do parto 

2) Grupos de referência:
Para cada dummy, retirar uma das categorias binárias como referência

3) Transformar a variável PARTO em binária:
1 = Normal, 0 = Cesáreo

4) Excluir do dataset:
4.1 - Todos os NaN
4.2 - Todas as linhas que tenham alguma informação "desconhecida"

5) Imprimir os resultados do modelo de regressão logística em termos dos coeficientes do modelo e Odds Ratio
